In [1]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.graph_stores.types import (
    EntityNode,
    KG_NODES_KEY,
    KG_RELATIONS_KEY,
    Relation,
)
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import  HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.mistralai import MistralAI
from llama_index.llms.gemini import Gemini

from llama_index.core import  VectorStoreIndex, StorageContext, QueryBundle, load_index_from_storage, Settings, Document

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
Gemini

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
import json

load_dotenv(r"C:\Users\leduc\OneDrive\Desktop\NLP\grab-capstone-project\NavigaTech\AI_modules\.env")

True

In [ ]:
with open(r"C:\Users\leduc\OneDrive\Desktop\NLP\grab-capstone-project\NavigaTech\AI_modules\data\rag_data\sample_data.json") as f:
    data = json.load(f)

df = pd.DataFrame(data)
# df = df[0:100]
role = df["search_keyword"].str.upper().str.replace(' ', '_').unique()
job_title = df["title"].str.upper().str.replace(' ', '_').unique()
location = df["location"].str.upper().str.split(",").str[0].str.strip().str.replace(' ', '_').unique()



In [13]:
# llm = HuggingFaceInferenceAPI(model_name="mistralai/Mistral-7B-Instruct-v0.3", token=os.environ["HF_TOKEN"])
llm = MistralAI(model="mistral-large-lastest", api_key=os.environ["MISTRAL_API_KEY"], random_seed=221, max_tokens=1000)
embbeding_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

Settings.llm = llm
Settings.embed_model = embbeding_model

In [5]:
graph_store = Neo4jPropertyGraphStore(
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    url="neo4j+s://2f233f79.databases.neo4j.io",
)

In [6]:
documents = []
for index, row in df.iterrows():
    meta = {
        "id": row["id"],
        "title": row["title"],
        "company": row["company"],
        "link": row["link"],
        "keyword": row["search_keyword"],
        "location": row["location"]
    }
    doc = Document(text = row["description"], metadata=meta)
    documents.append(doc)
len(documents)

100

In [ ]:
index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=embbeding_model,
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=llm
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

Generating embeddings:   0%|          | 0/13 [00:00<?, ?it/s]